In [ ]:
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
# calls_df = pd.read_csv('/home/alvaro/culebra_research_results/SOL-USD-170-C_2025-05-15.csv')
# puts_df = pd.read_csv('/home/alvaro/culebra_research_results/SOL-USD-180-P_2025-05-15.csv')

calls_df = pd.read_csv('/home/alvaro/culebra_research_results/SOL-USD-160-C_2025-05-15.csv')
puts_df = pd.read_csv('/home/alvaro/culebra_research_results/SOL-USD-180-P_2025-05-15.csv')

In [ ]:
prep_calls_df = calls_df[['symbol', 'ask', 'bid', 'mark_price', 'underlying_price', 'delta', 'funding_rate','created_at']]
prep_calls_df = prep_calls_df.copy()
prep_calls_df.loc[:, 'created_at'] = pd.to_datetime(prep_calls_df['created_at'], unit='ms').astype('int64')
prep_calls_df.loc[:, 'mid_price'] = (prep_calls_df['bid'] + prep_calls_df['ask']) / 2
prep_calls_df.loc[:, 'spread'] = abs(prep_calls_df['bid'] - prep_calls_df['ask']) / prep_calls_df['mid_price']

prep_puts_df = puts_df[['symbol', 'ask', 'bid', 'mark_price', 'underlying_price', 'delta', 'funding_rate', 'created_at']]
prep_puts_df = prep_puts_df.copy()
prep_puts_df.loc[:, 'created_at'] = pd.to_datetime(prep_puts_df['created_at'], unit='ms').astype('int64')
prep_puts_df.loc[:, 'mid_price'] = (prep_puts_df['bid'] + prep_puts_df['ask']) / 2
prep_puts_df.loc[:, 'spread'] = abs(prep_puts_df['bid'] - prep_puts_df['ask']) / prep_puts_df['mid_price']



In [ ]:
prep_puts_df['pos'] = 1.0 / abs(prep_puts_df['delta'])
prep_calls_df['pos'] = 1.0 / abs(prep_calls_df['delta'])

prep_puts_df['ret'] = abs(prep_puts_df['pos']) * prep_puts_df['funding_rate'] * prep_puts_df['underlying_price'] * 15 / (8 *3600)
prep_calls_df['ret'] = abs(prep_calls_df['pos']) * prep_puts_df['funding_rate'] * prep_calls_df['underlying_price'] * 15 / (8 *3600)

prep_puts_df['leg_value'] = prep_puts_df['pos'] * prep_puts_df['mid_price']
prep_calls_df['leg_value'] = prep_calls_df['pos'] * prep_calls_df['mid_price']

In [ ]:
prep_puts_df = prep_puts_df.rename(columns=lambda x: 'put_' + x)
prep_calls_df = prep_calls_df.rename(columns=lambda x: 'call_' + x)

portfolio_df =  pd.merge(prep_puts_df, prep_calls_df, how='inner', left_index=True, right_index=True,)
portfolio_df['portfolio_value'] = portfolio_df['call_leg_value'] + portfolio_df['put_leg_value']
portfolio_df['portfolio_ret'] = portfolio_df['call_ret'] + portfolio_df['put_ret']
portfolio_df['cum_portfolio_ret'] = portfolio_df['portfolio_ret'].cumsum()
# portfolio_df

In [ ]:
plt.figure(figsize=(10, 3))  # Optional: Set the size of the plot
plt.plot(portfolio_df['call_underlying_price'], linestyle='-', color='r', label='Mid Price')
# Add labels and title
plt.xlabel('Index')
plt.ylabel('Price')
plt.title('Spot price over Time')
plt.legend()

# Show the plot
plt.show()

In [ ]:
plt.figure(figsize=(10, 3))  # Optional: Set the size of the plot
plt.plot(portfolio_df['call_leg_value'], linestyle='-', color='g', label='Call leg value')
plt.plot(portfolio_df['put_leg_value'], linestyle='-', color='r', label='Put leg value')
# Add labels and title
plt.xlabel('Index')
plt.ylabel('Value')
plt.title('Legs values over Time')
plt.legend()

# Show the plot
plt.show()

In [ ]:
plt.figure(figsize=(10, 3))  # Optional: Set the size of the plot
plt.plot(portfolio_df['portfolio_value'], linestyle='-', color='b', label='Mid Price')

# Add labels and title
plt.xlabel('Index')
plt.ylabel('Value')
plt.title('Portfolio Value')
plt.legend()

# Show the plot
plt.show()

In [ ]:
plt.figure(figsize=(10, 3))  # Optional: Set the size of the plot
plt.plot(portfolio_df['portfolio_ret'], linestyle='-', color='b', label='Return')

# Add labels and title
plt.xlabel('Index')
plt.ylabel('Value')
plt.title('Portfolio Return')
plt.legend()

# Show the plot
plt.show()

In [ ]:
plt.figure(figsize=(10, 3))  # Optional: Set the size of the plot
plt.plot(portfolio_df['cum_portfolio_ret'], linestyle='-', color='b', label='Return')

# Add labels and title
plt.xlabel('Index')
plt.ylabel('Value')
plt.title('Cumulative Portfolio Return')
plt.legend()

# Show the plot
plt.show()